In [367]:
import pandas as pd
import numpy as np

In [368]:
Vdd = 0.8

In [369]:
def is_on_or_off(mostype, Vgate):
    if mostype == 'nmos':
        if Vgate == 0:
            return 'off'
        else:
            return 'on'
    elif mostype == 'pmos':
        if Vgate == Vdd:
            return 'off'
        else:
            return 'on'

In [370]:
def get_single_mos_leakage(mostype, Vdrain, Vgate, Vsource, width):
    if Vdrain == 1:
        Vdrain = Vdd
    if Vgate == 1:
        Vgate = Vdd
    if Vsource == 1:
        Vsource = Vdd
    filename = f"./single-mos/{mostype}_{is_on_or_off(mostype, Vgate)}_{width}W.txt"
    header = ["Temp", "Vdrain", "Vgate", "Vsource", "Vbody", "Id", "Ig", "Is", "Ib"]
    
    # read the file line by line and append it to a df with the header
    with open(filename, 'r') as f:
        lines = f.readlines()
        data = []
        for line in lines:
            data.append(line.split())

    df = pd.DataFrame(data, columns=header)
    df = df.astype(float)
    
    # return Id, Ig, Is, Ib for the given Vdrain, Vgate, Vsource
    currents = df[(df['Vdrain'] == Vdrain) & (df['Vgate'] == Vgate) & (df['Vsource'] == Vsource)][['Id', 'Ig', 'Is', 'Ib']].values[0]
    return [abs(current) for current in currents]

In [371]:
def get_intermediate_voltage(mostype, in1, in2):
    filename = f"./mos-stack/{mostype}_stack_{in1}{in2}.txt"
    with open(filename, 'r') as f:
        lines = f.readlines()
        Vint = lines[0].split()[3]
    return Vint

In [372]:
def nand_leakage(in1, in2):
    # if in1 == 0 and in2 == 0:
    out = 0 if in1 == 1 and in2 == 1 else Vdd
    Vint = get_intermediate_voltage('nmos', in1, in2)
    Vint = float(Vint)
    print(f"Vint: {Vint}")
    # round Vint to the nearest multiple of 0.05
    Vint = round(Vint * 20) / 20
    print(f"Vint: {Vint}")
    leak_pmos_1 = get_single_mos_leakage('pmos', Vdd, in1, out, 2)
    leak_pmos_2 = get_single_mos_leakage('pmos', Vdd, in2, out, 2)
    leak_nmos_1 = get_single_mos_leakage('nmos', out, in1, Vint, 1)
    leak_nmos_2 = get_single_mos_leakage('nmos', Vint, in2, 0, 1)

    print(f"leak_pmos_1: {leak_pmos_1}")
    print(f"leak_pmos_2: {leak_pmos_2}")
    print(f"leak_nmos_1: {leak_nmos_1}")
    print(f"leak_nmos_2: {leak_nmos_2}")

    # print(leak_nmos_2.sum())
    # supply_leakage = (leak_nmos_2[2] + leak_nmos_2[1] + leak_nmos_2[3] # Is, Ig, Ib of nmos2
    #                     + leak_nmos_1[1] + leak_nmos_1[3] # Ig, Ib of nmos1
    #                     + leak_pmos_1[1] + leak_pmos_1[3] # Ig, Ib of pmos1
    #                     + leak_pmos_2[1] + leak_pmos_2[3]) # Ig, Ib of pmos2

    if in1 == 0 and in2 == 0:
        supply_leakage = (leak_pmos_1[1] + leak_pmos_1[3] + leak_pmos_2[1] + leak_pmos_2[3]
                        #   + leak_nmos_1[1] + leak_nmos_1[3] + leak_nmos_2[1] + leak_nmos_2[3] + leak_nmos_2[2])
                          + leak_nmos_1[0])
    elif in1 == 0 and in2 == 1:
        supply_leakage = (leak_pmos_1[1] + leak_pmos_1[3] + leak_pmos_2[0] + leak_nmos_1[0])
    elif in1 == 1 and in2 == 0:
        supply_leakage = (leak_pmos_1[0] + leak_pmos_2[1] + leak_pmos_2[3] + leak_nmos_1[1] + leak_nmos_1[3] + leak_nmos_2[0])
    elif in1 == 1 and in2 == 1:
        supply_leakage = (leak_pmos_1[0] + leak_pmos_2[0])
    
    # if in1 == 0 and in2 == 0:
    #     total_leakage = supply_leakage
    # elif in1 == 0 and in2 == 1:
    #     total_leakage = supply_leakage + leak_nmos_2[1] + leak_pmos_2[1]
    # elif in1 == 1 and in2 == 0:
    #     total_leakage = supply_leakage + leak_nmos_1[1] + leak_pmos_1[1]
    # else:
    #     total_leakage = supply_leakage + leak_nmos_1[1] + leak_nmos_2[1] + leak_pmos_1[1] + leak_pmos_2[1]

    gate_leakage = leak_pmos_1[1] + leak_pmos_2[1] + leak_nmos_1[1] + leak_nmos_2[1]
    # if in1 == 0 and in2 == 0:
    #     gate_leakage = leak_pmos_1[1] + leak_pmos_2[1]
    # elif in1 == 0 and in2 == 1:
    #     gate_leakage = leak_nmos_2[1] + leak_pmos_1[1]
    # elif in1 == 1 and in2 == 0:
    #     gate_leakage = leak_nmos_1[1] + leak_pmos_2[1]
    # elif in1 == 1 and in2 == 1:
    #     gate_leakage = leak_nmos_1[1] + leak_nmos_2[1]

    return np.asarray((supply_leakage, gate_leakage))

In [389]:
nand_leakage(1,1)

Vint: 2.27848203e-05
Vint: 0.0
leak_pmos_1: [1.51874542e-08, 3.0271714e-09, 1.82146252e-08, 1.56125113e-16]
leak_pmos_2: [1.51874542e-08, 3.0271714e-09, 1.82146252e-08, 1.56125113e-16]
leak_nmos_1: [2.48207884e-09, 4.97313923e-09, 2.48207884e-09, 8.98154406e-12]
leak_nmos_2: [2.48207884e-09, 4.97313923e-09, 2.48207884e-09, 8.98154406e-12]


array([3.03749084e-08, 1.60006213e-08])

In [390]:
def nor_leakage(in1, in2):
    out = Vdd if in1 == 0 and in2 == 0 else 0
    Vint = get_intermediate_voltage('pmos', in1, in2)
    Vint = float(Vint)
    print(f"Vint: {Vint}")
    # round Vint to the nearest multiple of 0.05
    Vint = round(Vint * 20) / 20
    print(f"Vint: {Vint}")
    leak_pmos_1 = get_single_mos_leakage('pmos', Vdd, in1, Vint, 3)
    leak_pmos_2 = get_single_mos_leakage('pmos', Vint, in2, out, 3)
    leak_nmos_1 = get_single_mos_leakage('nmos', out, in1, 0, 1)
    leak_nmos_2 = get_single_mos_leakage('nmos', out, in2, 0, 1)

    print(f"leak_pmos_1: {leak_pmos_1}")
    print(f"leak_pmos_2: {leak_pmos_2}")
    print(f"leak_nmos_1: {leak_nmos_1}")
    print(f"leak_nmos_2: {leak_nmos_2}")

    # print(leak_nmos_2.sum())
    # supply_leakage = (leak_nmos_2[2] + leak_nmos_2[1] + leak_nmos_2[3] # Is, Ig, Ib of nmos2
    #                     + leak_nmos_1[2] + leak_nmos_1[1] + leak_nmos_1[3] # Is, Ig, Ib of nmos1
    #                     + leak_pmos_1[1] + leak_pmos_1[3] # Ig, Ib of pmos1
    #                     + leak_pmos_2[1] + leak_pmos_2[3]) # Ig, Ib of pmos2

    if in1 == 0 and in2 == 0:
        supply_leakage = (leak_pmos_1[1] + leak_pmos_1[3] + leak_pmos_2[1] + leak_pmos_2[3]
                          + leak_nmos_1[0] + leak_nmos_2[0])
    elif in1 == 0 and in2 == 1:
        supply_leakage = leak_pmos_1[1] + leak_pmos_1[3] + leak_pmos_2[0]
    elif in1 == 1 and in2 == 0:
        supply_leakage = leak_pmos_1[0] 
    elif in1 == 1 and in2 == 1:
        supply_leakage = leak_pmos_1[0]
    
    # if in1 == 0 and in2 == 0:
    #     total_leakage = supply_leakage
    # elif in1 == 0 and in2 == 1:
    #     total_leakage = supply_leakage + leak_nmos_2[1] + leak_pmos_2[1]
    # elif in1 == 1 and in2 == 0:
    #     total_leakage = supply_leakage + leak_nmos_1[1] + leak_pmos_1[1]
    # else:
    #     total_leakage = supply_leakage + leak_nmos_1[1] + leak_nmos_2[1] + leak_pmos_1[1] + leak_pmos_2[1]

    gate_leakage = leak_pmos_1[1] + leak_pmos_2[1] + leak_nmos_1[1] + leak_nmos_2[1]
    # if in1 == 0 and in2 == 0:
    #     gate_leakage = leak_pmos_1[1] + leak_pmos_2[1]
    # elif in1 == 0 and in2 == 1:
    #     gate_leakage = leak_nmos_2[1] + leak_pmos_1[1]
    # elif in1 == 1 and in2 == 0:
    #     gate_leakage = leak_nmos_1[1] + leak_pmos_2[1]
    # elif in1 == 1 and in2 == 1:
    #     gate_leakage = leak_nmos_1[1] + leak_nmos_2[1]

    return np.asarray((supply_leakage, gate_leakage))

In [394]:
nor_leakage(0,0)

Vint: 0.799916841
Vint: 0.8
leak_pmos_1: [1.1359842e-08, 2.29137893e-08, 1.1359842e-08, 1.94105239e-10]
leak_pmos_2: [1.1359842e-08, 2.29137893e-08, 1.1359842e-08, 1.94105239e-10]
leak_nmos_1: [5.90742119e-09, 1.57897691e-09, 4.32844424e-09, 4.0224803e-17]
leak_nmos_2: [5.90742119e-09, 1.57897691e-09, 4.32844424e-09, 4.0224803e-17]


array([5.80306315e-08, 4.89855324e-08])

In [395]:
def not_leakage(in1):
    out = Vdd if in1 == 0 else 0
    leak_pmos = get_single_mos_leakage('pmos', Vdd, in1, out, 2)
    leak_nmos = get_single_mos_leakage('nmos', out, in1, 0, 1)

    print(f"leak_pmos: {leak_pmos}")
    print(f"leak_nmos: {leak_nmos}")

    supply_leakage = leak_nmos[0] + leak_pmos[1] + leak_pmos[3] if in1 == 0 else leak_pmos[0]

    # supply_leakage = (leak_nmos[2] + leak_nmos[1] + leak_nmos[3] # Is, Ig, Ib of nmos
    #                     + leak_pmos[1] + leak_pmos[3] # Ig, Ib of pmos
    # )
    # supply_leakage = (leak_pmos[0] + leak_pmos[1] + leak_pmos[3]
    #                   + leak_nmos[1] + leak_nmos[3])
    # if in1 == 0:
    #     total_leakage = supply_leakage
    # else:
    #     total_leakage = supply_leakage + leak_nmos[1] + leak_pmos[1]

    gate_leakage = leak_pmos[1] + leak_nmos[1]
    # if in1 == 0:
    #     gate_leakage = leak_pmos[1]
    # elif in1 == 1:
    #     gate_leakage = abs(leak_pmos[1]) + abs(leak_nmos[1])

    return np.asarray((supply_leakage, gate_leakage))

In [398]:
not_leakage(1)

leak_pmos: [1.51874542e-08, 3.0271714e-09, 1.82146252e-08, 1.56125113e-16]
leak_nmos: [2.48207884e-09, 4.97313923e-09, 2.48207884e-09, 8.98154406e-12]


array([1.51874542e-08, 8.00031063e-09])

In [378]:
def invert(in1):
    return 0 if in1 == 1 else 1

In [379]:
def CLA(P0, P1, P2, P3, G0, G1, G2, G3, Cn):
    K1 = invert(Cn)
    K2 = P0 & G0
    K3 = G0 & K1
    Cnx = invert(K2 | K3)

    K4 = P1 & G1
    K5 = K2 & G1
    K6 = K3 & G1
    K7 = K4 | K5
    Cny = invert(K6 | K7)

    K8 = P2 & G2
    K9 = K4 & G2
    K10 = K5 & G2
    K11 = K6 & G2
    K12 = K8 | K9
    K13 = K10 | K11
    Cnz = invert(K12 | K13)

    K14 = P3 & G3
    K15 = K8 & G3
    K16 = K9 & G3
    K17 = G0 & G1
    K18 = G2 & G3
    K19 = K17 & K18
    K20 = K14 | K15
    K21 = K16 | K19
    G = K20 | K21

    K22 = P0 | P1
    K23 = P2 | P3
    P = K22 | K23

    return Cnx, Cny, Cnz, G, P

In [380]:
CLA(0,0,0,0,0,0,0,0,0)

(1, 1, 1, 0, 0)

In [381]:
def CLA_test(P0, P1, P2, P3, G0, G1, G2, G3, Cn):
    P = P0 | P1 | P2 | P3
    G = (G0&G1&G2&G3) | (P1&G1&G2&G3) | (P2&G2&G3) | (P3&G3)
    K = invert(Cn)
    Cnz = invert((G2&G1&G0&K) | (P0&G2&G1&G0) | (P1&G2&G1) | (P2&G2))
    Cny = invert((G1&G0&K) | (P0&G1&G0) | (P1&G1))
    Cnx = invert((G0&K) | (P0&G0))
    return Cnx, Cny, Cnz, G, P

In [382]:
P0, P1, P2, P3 = 0, 0, 0, 0
G0, G1, G2, G3 = 0, 1, 0, 0
Cn = 0
print(CLA(P0, P1, P2, P3, G0, G1, G2, G3, Cn))
print(CLA_test(P0, P1, P2, P3, G0, G1, G2, G3, Cn))

(1, 1, 1, 0, 0)
(1, 1, 1, 0, 0)


In [383]:
def CLA_leakage(P0, P1, P2, P3, G0, G1, G2, G3, Cn):
    K1 = invert(Cn)
    K2 = P0 & G0
    K3 = G0 & K1
    Cnx = invert(K2 | K3)

    K4 = P1 & G1
    K5 = K2 & G1
    K6 = K3 & G1
    K7 = K4 | K5
    Cny = invert(K6 | K7)

    K8 = P2 & G2
    K9 = K4 & G2
    K10 = K5 & G2
    K11 = K6 & G2
    K12 = K8 | K9
    K13 = K10 | K11
    Cnz = invert(K12 | K13)

    K14 = P3 & G3
    K15 = K8 & G3
    K16 = K9 & G3
    K17 = G0 & G1
    K18 = G2 & G3
    K19 = K17 & K18
    K20 = K14 | K15
    K21 = K16 | K19
    G = K20 | K21

    K22 = P0 | P1
    K23 = P2 | P3
    P = K22 | K23

    leakage = (  not_leakage(Cn) # K1 = ~Cn
                + nand_leakage(P0, G0) + not_leakage(invert(P0 & G0)) # K2 = P0 & G0
                + nand_leakage(G0, K1) + not_leakage(invert(G0 & K1)) # K3 = G0 & K1
                + nor_leakage(K2, K3) # Cnx = ~(K2 | K3)

                + nand_leakage(P1, G1) + not_leakage(invert(P1 & G1)) # K4 = P1 & G1
                + nand_leakage(K2, G1) + not_leakage(invert(K2 & G1)) # K5 = K2 & G1
                + nand_leakage(K3, G1) + not_leakage(invert(K3 & G1)) # K6 = K3 & G1
                + nor_leakage(K4, K5) + not_leakage(invert(K4 | K5)) # K7 = K4 | K5
                + nor_leakage(K6, K7) # Cny = ~(K6 | K7)

                + nand_leakage(P2, G2) + not_leakage(invert(P2 & G2)) # K8 = P2 & G2
                + nand_leakage(K4, G2) + not_leakage(invert(K4 & G2)) # K9 = K4 & G2
                + nand_leakage(K5, G2) + not_leakage(invert(K5 & G2)) # K10 = K5 & G2
                + nand_leakage(K6, G2) + not_leakage(invert(K6 & G2)) # K11 = K6 & G2
                + nor_leakage(K8, K9) + not_leakage(invert(K8 | K9)) # K12 = K8 | K9
                + nor_leakage(K10, K11) + not_leakage(invert(K10 | K11)) # K13 = K10 | K11
                + nor_leakage(K12, K13) # Cnz = ~(K12 | K13)

                + nand_leakage(P3, G3) + not_leakage(invert(P3 & G3)) # K14 = P3 & G3
                + nand_leakage(K8, G3) + not_leakage(invert(K8 & G3)) # K15 = K8 & G3
                + nand_leakage(K9, G3) + not_leakage(invert(K9 & G3)) # K16 = K9 & G3
                + nand_leakage(G0, G1) + not_leakage(invert(G0 & G1)) # K17 = G0 & G1
                + nand_leakage(G2, G3) + not_leakage(invert(G2 & G3)) # K18 = G2 & G3
                + nand_leakage(K17, K18) + not_leakage(invert(K17 & K18)) # K19 = K17 & K18
                + nor_leakage(K14, K15) + not_leakage(invert(K14 | K15)) # K20 = K14 | K15
                + nor_leakage(K16, K19) + not_leakage(invert(K16 | K19)) # K21 = K16 | K19
                + nor_leakage(K20, K21) + not_leakage(invert(K20 | K21)) # G = K20 | K21

                + nor_leakage(P0, P1) + not_leakage(invert(P0 | P1)) # K22 = P0 | P1
                + nor_leakage(P2, P3) + not_leakage(invert(P2 | P3)) # K23 = P2 | P3
                + nor_leakage(K22, K23) + not_leakage(invert(K22 | K23)) # P = K22 | K23
    )

    return leakage


In [403]:
CLA_leakage(1,1,1,1,0,0,1,1,0)

leak_pmos: [6.89704701e-09, 1.39119436e-08, 6.89704701e-09, 1.17849609e-10]
leak_nmos: [5.90742119e-09, 1.57897691e-09, 4.32844424e-09, 4.0224803e-17]
Vint: 0.637399352
Vint: 0.65
leak_pmos_1: [1.73069799e-19, 1.77635684e-15, 1.73069818e-19, 1.04083409e-17]
leak_pmos_2: [6.89704701e-09, 1.39119436e-08, 6.89704701e-09, 1.17849609e-10]
leak_nmos_1: [1.97965718e-09, 1.4281909e-12, 1.9810651e-09, 2.02429541e-14]
leak_nmos_2: [3.23497929e-09, 8.99086974e-10, 2.3358923e-09, 1.73603792e-17]
leak_pmos: [1.51874542e-08, 3.0271714e-09, 1.82146252e-08, 1.56125113e-16]
leak_nmos: [2.48207884e-09, 4.97313923e-09, 2.48207884e-09, 8.98154406e-12]
Vint: 6.24381066e-05
Vint: 0.0
leak_pmos_1: [6.89704701e-09, 1.39119436e-08, 6.89704701e-09, 1.17849609e-10]
leak_pmos_2: [1.73069799e-19, 1.77635684e-15, 1.73069818e-19, 1.04083409e-17]
leak_nmos_1: [5.90742119e-09, 1.57897691e-09, 4.32844424e-09, 4.0224803e-17]
leak_nmos_2: [2.48207884e-09, 4.97313923e-09, 2.48207884e-09, 8.98154406e-12]
leak_pmos: [1.5187

array([1.13765985e-06, 9.16456797e-07])